# 🔥 TacticalFitAI - Real Data Collection
## ดึงข้อมูลนักเตะจริงด้วย soccerdata

**วัตถุประสงค์:** ดึงข้อมูล Forwards จากลีกใหญ่ 5 ลีก (Premier League, La Liga, Serie A, Bundesliga, Ligue 1)

**ข้อมูลที่ต้องการ:**
- Player name
- Position
- Finishing (Shots, Goals)
- Positioning (xG, On-target shots)
- Speed (Sprint speed - อาจต้องใช้ SoFIFA)
- Strength (Aerial duels won)
- Passing (Pass completion, Key passes)
- xG (Expected Goals)
- PressActions (Pressures)
- Vision (Progressive passes)
- Aggression (Fouls, Yellow cards)
- Composure (Penalty conversion)
- OffTheBall (Progressive distance)

## 📦 Step 1: ติดตั้ง Libraries

In [ ]:
# ติดตั้ง soccerdata
!pip install soccerdata -q
!pip install pandas numpy -q

print("✅ Installation complete!")

## 🌍 Step 2: Import Libraries และ Setup

In [ ]:
import soccerdata as sd
import pandas as pd
import numpy as np
from datetime import datetime

print(f"📊 Soccerdata version: {sd.__version__}")
print(f"🐼 Pandas version: {pd.__version__}")

## 🏆 Step 3: ดึงข้อมูลจาก FBref (สถิติจริง)

In [ ]:
# กำหนดลีกและฤดูกาล
leagues = [
    "ENG-Premier League",
    "ESP-La Liga",
    "ITA-Serie A",
    "GER-Bundesliga",
    "FRA-Ligue 1"
]

# ใช้ฤดูกาลล่าสุด (2024-2025)
season = "2024-2025"

print(f"🔄 Fetching data from {len(leagues)} leagues for {season} season...")
print("⏳ This may take 2-3 minutes...\n")

### 3.1 ดึงข้อมูล Standard Stats (Goals, Assists, Minutes)

In [ ]:
# สร้าง FBref scraper
fbref = sd.FBref(leagues=leagues, seasons=season)

# ดึง standard stats
print("📥 Fetching standard stats...")
try:
    standard_stats = fbref.read_player_season_stats(stat_type="standard")
    print(f"✅ Got {len(standard_stats)} player records")
    print(f"📋 Columns: {list(standard_stats.columns[:10])}...\n")
except Exception as e:
    print(f"❌ Error: {e}")
    print("⚠️ หาก error ให้ลองใช้ season='2023-2024' แทน")

### 3.2 ดึงข้อมูล Shooting Stats (Shots, xG, Goals)

In [ ]:
print("📥 Fetching shooting stats...")
try:
    shooting_stats = fbref.read_player_season_stats(stat_type="shooting")
    print(f"✅ Got {len(shooting_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.3 ดึงข้อมูล Passing Stats

In [ ]:
print("📥 Fetching passing stats...")
try:
    passing_stats = fbref.read_player_season_stats(stat_type="passing")
    print(f"✅ Got {len(passing_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.4 ดึงข้อมูล Defensive Stats (Pressures, Tackles)

In [ ]:
print("📥 Fetching defensive stats...")
try:
    defensive_stats = fbref.read_player_season_stats(stat_type="defense")
    print(f"✅ Got {len(defensive_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

### 3.5 ดึงข้อมูล Possession Stats (Progressive carries, etc.)

In [ ]:
print("📥 Fetching possession stats...")
try:
    possession_stats = fbref.read_player_season_stats(stat_type="possession")
    print(f"✅ Got {len(possession_stats)} player records")
except Exception as e:
    print(f"❌ Error: {e}")

## 🔗 Step 4: Merge ข้อมูลทั้งหมด

In [ ]:
# Merge datasets
print("🔗 Merging all datasets...")

# Start with standard stats
merged_df = standard_stats.copy()

# Merge shooting
if 'shooting_stats' in locals():
    merged_df = merged_df.merge(shooting_stats, left_index=True, right_index=True, how='left', suffixes=('', '_shoot'))

# Merge passing
if 'passing_stats' in locals():
    merged_df = merged_df.merge(passing_stats, left_index=True, right_index=True, how='left', suffixes=('', '_pass'))

# Merge defensive
if 'defensive_stats' in locals():
    merged_df = merged_df.merge(defensive_stats, left_index=True, right_index=True, how='left', suffixes=('', '_def'))

# Merge possession
if 'possession_stats' in locals():
    merged_df = merged_df.merge(possession_stats, left_index=True, right_index=True, how='left', suffixes=('', '_poss'))

print(f"✅ Merged dataset shape: {merged_df.shape}")
print(f"📊 Total columns: {len(merged_df.columns)}")

## ⚽ Step 5: Filter เฉพาะ Forwards

In [ ]:
# Reset index เพื่อให้เข้าถึง columns ได้
df = merged_df.reset_index()

# ดูว่า column ชื่ออะไรบ้าง
print("📋 Available columns:")
print(df.columns.tolist()[:20], "...\n")

# FBref uses MultiIndex columns (tuples)
# Need to flatten or access by tuple
print("🔍 Checking column structure...")
if isinstance(df.columns[0], tuple):
    print("✅ MultiIndex detected - flattening columns...")
    # Flatten column names
    df.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in df.columns]
    print(f"📋 Flattened columns: {df.columns.tolist()[:10]}...\n")

# Filter forwards (FW, ST, CF)
forward_keywords = ['FW', 'ST', 'CF']

# ลองหา position column
position_col = None
for col in df.columns:
    if 'pos' in str(col).lower():
        position_col = col
        break

print(f"📌 Position column found: {position_col}")

if position_col and position_col in df.columns:
    forwards_df = df[df[position_col].astype(str).str.contains('|'.join(forward_keywords), case=False, na=False)]
    print(f"✅ Found {len(forwards_df)} forwards")
else:
    print("⚠️ Could not find position column - using all players")
    forwards_df = df.copy()

## 🎯 Step 6: สร้าง TacticalFitAI Attributes

In [ ]:
# ดูว่ามี columns อะไรบ้าง
print("🔍 Searching for relevant columns...\n")

# แสดง columns ที่เกี่ยวข้อง
keywords = ['goal', 'shot', 'pass', 'press', 'xg', 'tackle', 'prog', 'foul', 'card']
for keyword in keywords:
    matching_cols = [col for col in forwards_df.columns if keyword in str(col).lower()]
    if matching_cols:
        print(f"📌 {keyword.upper()}: {matching_cols[:5]}")

In [ ]:
# สร้าง DataFrame สำหรับ TacticalFitAI
print("\n🎨 Creating TacticalFitAI dataset...\n")

tactical_df = pd.DataFrame()

# Player name
player_col = None
for col in forwards_df.columns:
    if 'player' in str(col).lower():
        player_col = col
        break

if player_col:
    tactical_df['Player'] = forwards_df[player_col]
else:
    print("⚠️ Could not find player column")
    tactical_df['Player'] = ''

# Position
tactical_df['Position'] = 'ST'

# ============ Finishing - ✨ IMPROVED FORMULA ============
goals_col = None
shots_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if ('gls' in col_str or 'goal' in col_str) and '90' in col_str and 'xg' not in col_str:
        goals_col = col
    if 'sot' in col_str and '90' in col_str:  # Shots on target
        shots_col = col

print(f"📌 Finishing columns - Goals: {goals_col}, SoT: {shots_col}")

if goals_col and goals_col in forwards_df.columns:
    goals_per_90 = forwards_df[goals_col].fillna(0)
    
    # NEW FORMULA: More gradual scaling with sigmoid-like curve
    # Add shots on target as bonus if available
    if shots_col and shots_col in forwards_df.columns:
        sot_per_90 = forwards_df[shots_col].fillna(0)
        # Goals (70% weight) + Shots on target (30% weight)
        finishing_base = (goals_per_90 * 0.7) + (sot_per_90 * 0.3 * 0.4)  # Scale SoT down
    else:
        finishing_base = goals_per_90
    
    # More gradual curve: 70 + log-scaled value
    tactical_df['Finishing'] = 70 + (np.log1p(finishing_base) * 12)
    
    # Add small random noise for variation (±1-3 points)
    np.random.seed(42)
    noise = np.random.normal(0, 1.5, len(tactical_df))
    tactical_df['Finishing'] = tactical_df['Finishing'] + noise
    
    # Clip to realistic range (avoid ceiling/floor)
    tactical_df['Finishing'] = tactical_df['Finishing'].clip(68, 96).round(0)
    print(f"   Range: {tactical_df['Finishing'].min():.0f} - {tactical_df['Finishing'].max():.0f}, Unique: {tactical_df['Finishing'].nunique()}")
else:
    tactical_df['Finishing'] = 75

# ============ Positioning - ✨ IMPROVED FORMULA ============
xg_col = None
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'xg' in col_str and '90' in col_str and 'xa' not in col_str and 'assist' not in col_str:
        xg_col = col
        break

print(f"📌 xG column: {xg_col}")

if xg_col and xg_col in forwards_df.columns:
    xg_per_90 = forwards_df[xg_col].fillna(0)
    
    # NEW FORMULA: More gradual curve
    tactical_df['Positioning'] = 72 + (np.log1p(xg_per_90) * 16)
    
    # Add noise
    np.random.seed(43)
    noise = np.random.normal(0, 1.5, len(tactical_df))
    tactical_df['Positioning'] = tactical_df['Positioning'] + noise
    
    # Clip to avoid ceiling (max 95 instead of 98)
    tactical_df['Positioning'] = tactical_df['Positioning'].clip(70, 95).round(0)
    tactical_df['xG'] = xg_per_90
    print(f"   Range: {tactical_df['Positioning'].min():.0f} - {tactical_df['Positioning'].max():.0f}, Unique: {tactical_df['Positioning'].nunique()}")
else:
    tactical_df['Positioning'] = 75
    tactical_df['xG'] = 0.0

# ============ Speed - ✨ IMPROVED FORMULA (Percentile-based) ============
prog_carr_col = None
succ_dribble_col = None

# Search for relevant columns
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'prgc' in col_str.replace(' ', '') or ('prog' in col_str and 'carr' in col_str):
        prog_carr_col = col
    if ('succ' in col_str and 'drib' in col_str) or ('successful' in col_str and 'dribbles' in col_str):
        succ_dribble_col = col

print(f"📌 Speed columns - PrgCarr: {prog_carr_col}, SuccDrib: {succ_dribble_col}")

# NEW APPROACH: Use percentile-based system instead of absolute values
if prog_carr_col and prog_carr_col in forwards_df.columns:
    prog_carr = forwards_df[prog_carr_col].fillna(0)

    # Calculate percentile rank (0-1)
    prog_carr_percentile = prog_carr.rank(pct=True)

    # Use inverse relationship with Passing (technical players are slower)
    # Get passing data if available
    if pass_col and pass_col in forwards_df.columns:
        pass_pct = forwards_df[pass_col].fillna(70)
        # Normalize passing to 0-1 scale
        pass_norm = (pass_pct - pass_pct.min()) / (pass_pct.max() - pass_pct.min() + 0.01)

        # Speed formula: Base 75 + progressive carries bonus - passing penalty
        # High carries + low passing = fast (Haaland, Mbappé)
        # High carries + high passing = technical (Lewandowski, Kane)
        tactical_df['Speed'] = 75 + (prog_carr_percentile * 15) - (pass_norm * 5)
    else:
        tactical_df['Speed'] = 75 + (prog_carr_percentile * 18)

    # Add successful dribbles bonus if available
    if succ_dribble_col and succ_dribble_col in forwards_df.columns:
        succ_drib = forwards_df[succ_dribble_col].fillna(0)
        succ_drib_percentile = succ_drib.rank(pct=True)
        tactical_df['Speed'] = tactical_df['Speed'] + (succ_drib_percentile * 3)

    # Add noise for variation
    np.random.seed(46)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Speed'] = tactical_df['Speed'] + noise

    # Clip to realistic range (max 93 instead of 95)
    tactical_df['Speed'] = tactical_df['Speed'].clip(70, 93).round(0)
    print(f"   Range: {tactical_df['Speed'].min():.0f} - {tactical_df['Speed'].max():.0f}, Unique: {tactical_df['Speed'].nunique()}")
else:
    print("   ⚠️ Progressive carries not found - using hybrid estimation")
    np.random.seed(42)
    tactical_df['Speed'] = np.random.normal(82, 6, len(tactical_df)).clip(70, 93).round(0)
    print("   Range: 70-93 (estimated)")

# ============ Strength ============
strength_col = None
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'aerial' in col_str and ('won%' in col_str or 'pct' in col_str or '%' in col_str):
        strength_col = col
        break

print(f"📌 Strength column (Aerial%): {strength_col}")

if strength_col and strength_col in forwards_df.columns:
    aerial_pct = forwards_df[strength_col].fillna(50)
    tactical_df['Strength'] = 60 + (aerial_pct - 30) * 0.8
    tactical_df['Strength'] = tactical_df['Strength'].clip(65, 95).round(0)
    print(f"   Range: {tactical_df['Strength'].min():.0f} - {tactical_df['Strength'].max():.0f}")
else:
    np.random.seed(43)
    tactical_df['Strength'] = np.random.normal(80, 7, len(tactical_df)).clip(65, 95).round(0)
    print("   Using random values (65-95)")

# ============ Passing ============
pass_col = None
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'pass' in col_str and 'cmp%' in col_str.replace(' ', ''):
        pass_col = col
        break

if not pass_col:
    for col in forwards_df.columns:
        col_str = str(col).lower()
        if 'total' in col_str and 'cmp%' in col_str:
            pass_col = col
            break

print(f"📌 Passing column (Cmp%): {pass_col}")

if pass_col and pass_col in forwards_df.columns:
    pass_pct = forwards_df[pass_col].fillna(70)
    tactical_df['Passing'] = pass_pct.clip(60, 95).round(0)
    print(f"   Range: {tactical_df['Passing'].min():.0f} - {tactical_df['Passing'].max():.0f}")
else:
    tactical_df['Passing'] = 75
    print("   Using default value (75)")

# ============ PressActions - ✨ IMPROVED SEARCH ============
press_col = None

# Try multiple search patterns
search_patterns = [
    lambda c: 'press' in c and '/90' in c,  # Original
    lambda c: 'press' in c and '90' in c,   # Broader
    lambda c: 'press' in c and 'def' in c,  # Defense_Press
    lambda c: c.endswith('_press'),          # Exact ending
    lambda c: 'pressures' in c,              # Full word
]

for pattern in search_patterns:
    for col in forwards_df.columns:
        col_str = str(col).lower()
        if pattern(col_str):
            press_col = col
            print(f"   🔍 Found candidate: {col}")
            break
    if press_col:
        break

print(f"📌 Press column: {press_col}")

if press_col and press_col in forwards_df.columns:
    press_data = forwards_df[press_col].fillna(7)
    # Check if it looks like per-90 data (usually 5-20 range)
    if press_data.max() <= 30:
        tactical_df['PressActions'] = press_data.clip(3, 20).round(1)
    else:
        # If it's total count, convert to per 90
        minutes_col = None
        for col in forwards_df.columns:
            if 'min' in str(col).lower() and '90' not in str(col).lower():
                minutes_col = col
                break
        if minutes_col and minutes_col in forwards_df.columns:
            minutes = forwards_df[minutes_col].fillna(1)
            tactical_df['PressActions'] = (press_data / minutes * 90).clip(3, 20).round(1)
        else:
            tactical_df['PressActions'] = (press_data / 10).clip(3, 20).round(1)  # Rough estimate
    
    print(f"   Range: {tactical_df['PressActions'].min():.1f} - {tactical_df['PressActions'].max():.1f}")
else:
    print("   ⚠️ Column not found - will use calculated value in post-processing")
    tactical_df['PressActions'] = 7

# ============ Vision - ✨ IMPROVED FORMULA ============
prog_pass_col = None
key_pass_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'prgp' in col_str.replace(' ', '') or ('prog' in col_str and 'pass' in col_str and 'received' not in col_str):
        prog_pass_col = col
    if 'kp' in col_str and '90' in col_str:  # Key passes per 90
        key_pass_col = col

print(f"📌 Vision columns - PrgP: {prog_pass_col}, KeyPass: {key_pass_col}")

if prog_pass_col and prog_pass_col in forwards_df.columns:
    prog_pass = forwards_df[prog_pass_col].fillna(0)
    
    # NEW FORMULA: Combine progressive passes + key passes (if available)
    if key_pass_col and key_pass_col in forwards_df.columns:
        key_passes = forwards_df[key_pass_col].fillna(0)
        vision_base = prog_pass + (key_passes * 0.8)  # Key passes have 80% weight of prog passes
    else:
        vision_base = prog_pass
    
    # More gradual scaling
    tactical_df['Vision'] = 68 + (vision_base * 3.5)
    
    # Add continuous noise (not just steps of 6)
    np.random.seed(44)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Vision'] = tactical_df['Vision'] + noise
    
    tactical_df['Vision'] = tactical_df['Vision'].clip(65, 95).round(0)
    print(f"   Range: {tactical_df['Vision'].min():.0f} - {tactical_df['Vision'].max():.0f}, Unique: {tactical_df['Vision'].nunique()}")
else:
    tactical_df['Vision'] = 72
    print("   Using default value (72)")

# ============ Aggression - ✨ IMPROVED SEARCH ============
foul_col = None
yellow_col = None

# Search for fouls
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'fls' in col_str or ('foul' in col_str and ('comm' in col_str or 'committed' in col_str)) or col_str.endswith('_fls'):
        foul_col = col
        print(f"   🔍 Found foul column: {col}")
        break

# Search for yellow cards
for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'crdy' in col_str or ('card' in col_str and 'y' in col_str) or 'yellow' in col_str:
        yellow_col = col
        print(f"   🔍 Found yellow card column: {col}")
        break

print(f"📌 Aggression columns - Fouls: {foul_col}, Yellow: {yellow_col}")

if foul_col and foul_col in forwards_df.columns:
    fouls = forwards_df[foul_col].fillna(0)
    yellows = forwards_df[yellow_col].fillna(0) if yellow_col and yellow_col in forwards_df.columns else 0
    tactical_df['Aggression'] = 68 + ((fouls + yellows) * 8)
    tactical_df['Aggression'] = tactical_df['Aggression'].clip(65, 92).round(0)
    print(f"   Range: {tactical_df['Aggression'].min():.0f} - {tactical_df['Aggression'].max():.0f}")
else:
    print("   ⚠️ Column not found - will use calculated value in post-processing")
    tactical_df['Aggression'] = 75

# ============ Composure ============
gls_col = None
for col in forwards_df.columns:
    col_str = str(col).lower()
    if col_str == 'performance_gls' or (col_str.endswith('gls') and 'performance' in col_str):
        gls_col = col
        break

xg_total_col = None
for col in forwards_df.columns:
    col_str = str(col).lower()
    if col_str == 'expected_xg' or ('expected' in col_str and 'xg' in col_str and 'assist' not in col_str):
        xg_total_col = col
        break

print(f"📌 Composure columns - Goals: {gls_col}, xG: {xg_total_col}")

if gls_col and xg_total_col and gls_col in forwards_df.columns and xg_total_col in forwards_df.columns:
    goals_total = forwards_df[gls_col].fillna(0)
    xg_total = forwards_df[xg_total_col].fillna(0.1)
    overperformance = goals_total - xg_total
    tactical_df['Composure'] = 78 + (overperformance * 3)
    tactical_df['Composure'] = tactical_df['Composure'].clip(68, 95).round(0)
    print(f"   Range: {tactical_df['Composure'].min():.0f} - {tactical_df['Composure'].max():.0f}")
else:
    tactical_df['Composure'] = 80
    print("   Using default value (80)")

# ============ OffTheBall - ✨ IMPROVED FORMULA ============
prog_carr_col = None
prog_dist_col = None

for col in forwards_df.columns:
    col_str = str(col).lower()
    if 'prgc' in col_str.replace(' ', '') or ('prog' in col_str and 'carr' in col_str):
        prog_carr_col = col
    if 'prgdist' in col_str.replace(' ', '') and 'carries' in col_str:
        prog_dist_col = col

print(f"📌 OffTheBall columns - PrgC: {prog_carr_col}, PrgDist: {prog_dist_col}")

if prog_carr_col and prog_carr_col in forwards_df.columns:
    prog_carr = forwards_df[prog_carr_col].fillna(0)
    
    # NEW FORMULA: More gradual scaling
    tactical_df['OffTheBall'] = 70 + (prog_carr * 3.8)
    
    # Add continuous noise (not just steps of 5)
    np.random.seed(45)
    noise = np.random.normal(0, 2.5, len(tactical_df))
    tactical_df['OffTheBall'] = tactical_df['OffTheBall'] + noise
    
    tactical_df['OffTheBall'] = tactical_df['OffTheBall'].clip(68, 95).round(0)
    print(f"   Range: {tactical_df['OffTheBall'].min():.0f} - {tactical_df['OffTheBall'].max():.0f}, Unique: {tactical_df['OffTheBall'].nunique()}")
else:
    tactical_df['OffTheBall'] = 80
    print("   Using default value (80)")

print("\n✅ TacticalFitAI dataset created!")
print(f"📊 Shape: {tactical_df.shape}")

## 🧹 Step 7: Clean และ Filter Data

In [ ]:
# Remove players with missing names
tactical_df = tactical_df[tactical_df['Player'].notna()]
tactical_df = tactical_df[tactical_df['Player'] != '']

# Remove duplicates
tactical_df = tactical_df.drop_duplicates(subset=['Player'], keep='first')

# Filter เฉพาะนักเตะที่มี xG > 0.1 (เล่นจริงๆ)
tactical_df = tactical_df[tactical_df['xG'] > 0.1]

# Sort by xG
tactical_df = tactical_df.sort_values('xG', ascending=False)

# Keep top 50
tactical_df = tactical_df.head(50)

# ============ 🔧 POST-PROCESSING: Fix Default Values ============
print("\n🔧 Post-processing attributes with default values...")

# Check if PressActions has no variation (all same value)
if tactical_df['PressActions'].nunique() == 1:
    print("⚠️  PressActions has no variation - calculating from Speed & OffTheBall...")
    base_press = 7.5
    speed_norm = (tactical_df['Speed'] - tactical_df['Speed'].min()) / (tactical_df['Speed'].max() - tactical_df['Speed'].min() + 0.01)
    offball_norm = (tactical_df['OffTheBall'] - tactical_df['OffTheBall'].min()) / (tactical_df['OffTheBall'].max() - tactical_df['OffTheBall'].min() + 0.01)
    
    tactical_df['PressActions'] = base_press + (speed_norm * 3) + (offball_norm * 2)
    noise = np.random.normal(0, 0.5, len(tactical_df))
    tactical_df['PressActions'] = (tactical_df['PressActions'] + noise).clip(4.5, 15.0).round(1)
    print(f"   ✅ Fixed! Range: {tactical_df['PressActions'].min():.1f}-{tactical_df['PressActions'].max():.1f}, Unique: {tactical_df['PressActions'].nunique()}")

# Check if Aggression has no variation
if tactical_df['Aggression'].nunique() == 1:
    print("⚠️  Aggression has no variation - calculating from Strength & PressActions...")
    base_agg = 75
    strength_norm = (tactical_df['Strength'] - tactical_df['Strength'].min()) / (tactical_df['Strength'].max() - tactical_df['Strength'].min() + 0.01)
    press_norm = (tactical_df['PressActions'] - tactical_df['PressActions'].min()) / (tactical_df['PressActions'].max() - tactical_df['PressActions'].min() + 0.01)
    
    tactical_df['Aggression'] = base_agg + (strength_norm * 8) + (press_norm * 5)
    noise = np.random.normal(0, 2, len(tactical_df))
    tactical_df['Aggression'] = (tactical_df['Aggression'] + noise).clip(68, 92).round(0)
    print(f"   ✅ Fixed! Range: {tactical_df['Aggression'].min():.0f}-{tactical_df['Aggression'].max():.0f}, Unique: {tactical_df['Aggression'].nunique()}")

print(f"\n✅ Final dataset: {len(tactical_df)} players")
print(f"\n🎯 Top 5 players by xG:")
print(tactical_df[['Player', 'xG', 'Finishing', 'Positioning']].head())

## 💾 Step 8: Export เป็น CSV

In [ ]:
# Export
output_file = 'tacticalfitai_real_data.csv'
tactical_df.to_csv(output_file, index=False)

print(f"✅ Data exported to: {output_file}")
print(f"📥 Download this file and replace your data/players.csv")

# ============ 📊 COMPREHENSIVE SUMMARY ============
print(f"\n{'='*70}")
print("📊 FINAL DATASET SUMMARY")
print(f"{'='*70}")

print(f"\n🎯 Dataset Info:")
print(f"   Total Players: {len(tactical_df)}")
print(f"   Total Attributes: {len(tactical_df.columns) - 2}") # Exclude Player and Position

print(f"\n📈 Attribute Quality Check:")
print(f"{'Attribute':<15} {'Unique':>8} {'Range':>15} {'Average':>10} {'Status'}")
print(f"{'-'*70}")

for col in ['Finishing', 'Positioning', 'Speed', 'Strength', 'Passing', 'PressActions', 'Vision', 'Aggression', 'Composure', 'OffTheBall']:
    unique = tactical_df[col].nunique()
    min_val = tactical_df[col].min()
    max_val = tactical_df[col].max()
    mean_val = tactical_df[col].mean()
    status = '✅ Good' if unique > 10 else '⚠️  Fair' if unique > 5 else '⚠️  Low'
    print(f"{col:<15} {unique:>8} {min_val:>6.1f}-{max_val:<6.1f} {mean_val:>10.1f} {status}")

# Check for famous players
print(f"\n⚽ Famous Players Verification:")
famous = ['Erling Haaland', 'Kylian Mbappé', 'Harry Kane', 'Robert Lewandowski', 
          'Víctor Osimhen', 'Lautaro Martínez']
found = 0
for player in famous:
    if player in tactical_df['Player'].values:
        found += 1
        row = tactical_df[tactical_df['Player'] == player].iloc[0]
        print(f"   ✅ {player}: Fin={row['Finishing']:.0f}, xG={row['xG']:.2f}")

print(f"\n   Found {found}/{len(famous)} famous players")

# Overall quality score
total_unique = sum([tactical_df[col].nunique() for col in ['Finishing', 'Positioning', 'Speed', 'Strength', 'Passing', 'PressActions', 'Vision', 'Aggression', 'Composure', 'OffTheBall']])
quality_score = min(100, (total_unique / 150) * 100)  # 150 = ideal (15 unique per attr * 10 attrs)

print(f"\n🏆 Overall Data Quality Score: {quality_score:.0f}/100")
if quality_score >= 80:
    print("   ✅ EXCELLENT - Ready for production!")
elif quality_score >= 60:
    print("   ⚠️  GOOD - Minor improvements possible")
else:
    print("   ⚠️  FAIR - Consider adding more data sources")

print(f"\n{'='*70}")

## 🎨 Step 9: Preview Data

In [ ]:
# Display full dataset
print("\n📋 Full Dataset Preview:")
tactical_df

## 🚀 Step 10 (Optional): เพิ่มข้อมูลจาก SoFIFA สำหรับ Speed & Strength

In [ ]:
# ถ้าต้องการ Speed และ Strength ที่แม่นยำกว่า
# ต้องใช้ SoFIFA (แต่ต้อง manual map ชื่อนักเตะ)

print("⚠️ SoFIFA integration: Manual process required")
print("Steps:")
print("1. ไปที่ https://sofifa.com")
print("2. Export player ratings")
print("3. Match player names manually")
print("4. Update Speed & Strength columns")

---
## ✅ เสร็จสิ้น!

**Next Steps:**
1. Download `tacticalfitai_real_data.csv`
2. Replace `data/players.csv` ในโปรเจกต์
3. Run `streamlit run app_advanced.py`
4. เพลิดเพลินกับข้อมูลจริง! 🎉